In [1]:
#| default_exp kernels

In [2]:
#| export
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne
from functools import partial

from flax.struct import dataclass as flax_dataclass
from chewc.config import SimConfig
from chewc.structs import Population, SimParam

In [3]:
#| export
def compute_dosage(geno, is_active):
    """
    Computes dosage for all individuals, masking inactive ones.
    This is JIT-compatible because it preserves the array shape.
    """
    # Sum across the ploidy axis.
    # The output shape is (max_pop, n_chr, n_loci)
    dosage_all = jnp.sum(geno, axis=2)

    # Mask inactive individuals by setting their dosage to 0 (or another sentinel value).
    # The where clause ensures the function is traceable.
    masked_dosage = jnp.where(is_active[:, None, None], dosage_all, 0)
    
    # Reshape to (max_pop, n_loci_total)
    return masked_dosage.reshape(masked_dosage.shape[0], -1)

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()